# 함수 관계 정리 (report 생성 코드)

| 구분 | 순서 | 함수 이름 | 용도 / 설명 | 입력 | 출력 | 호출 관계 / 비고 |
|------|------|-----------|-------------|------|------|----------------|
| 데이터 전처리 | 1 | `attach_subtables` | MultiTable / MultiChart 블록에 **서브 블록(values)** 삽입 후, 최상위 blocks에서 제거 | `data` (dict) | 전처리된 `data` (dict) | `build_report` 실행 전에 선처리 |
| 문서 스타일 | 2 | `set_default_style` | Word 문서의 기본 스타일 지정:<br>폰트(`맑은 고딕`), 크기, 단락 간격, 페이지 크기(A4), 여백 | `doc` (Document) | 없음 | `build_report` 내부에서 최초 호출 |
| 문서 요소 | 3 | `add_block_heading` | 블록/섹션 제목 추가 (굵게, 크기 설정, 여백 포함) | `target` (Document/Cell), `text`, `font_name`, `font_size` | 생성된 Paragraph 객체 | `add_text_section`, `add_table_section`, `add_chart_section`, `multi_table_text`, `multi_table_chart` 등에서 호출 |
| 레이아웃 | 4 | `set_table_autofit` | 표의 너비 자동 맞춤 / 고정 폭 설정 | `table`, `autofit` (bool) | 없음 | `create_report_template` 내부에서 호출 |
| 레이아웃 | 5 | `create_report_template` | 보고서 레이아웃 템플릿 생성:<br>`1col`, `2col`, `full_table` 지원 | `doc`, `layout_type` | `Cell` 또는 `Cells` | `build_report`에서 블록 타입별 컨테이너 생성 시 사용 |
| 테이블 생성 | 6 | `create_table` | 지정한 크기의 빈 테이블 생성 | `doc`, `n_rows`, `n_cols`, `style` | Table 객체 | `multi_table_text`, `multi_table_chart` 등에서 호출 |
| 테이블 생성 | 7 | `render_table` | 리스트[dict] 데이터를 Word 표로 렌더링:<br>헤더(회색, 굵게, 가운데 정렬) + 데이터 | `values`, `container`, `style` | Table 객체 | `add_table_section`, `multi_table_chart` 내부에서 호출 |
| 블록 처리 | 8 | `add_text_section` | Text 블록 처리: 제목 + 본문 텍스트 (굵게 마크업 `**` 처리) | `doc`, `block`, `container` | 없음 | `build_report` → Text 블록 시 호출 |
| 블록 처리 | 9 | `add_table_section` | Table 블록 처리: 제목 + 표 렌더링 | `doc`, `block`, `container` | 없음 | `build_report` → Table 블록 시 호출 |
| 블록 처리 | 10 | `add_chart_section` | Chart 블록 처리: 제목 + 코드 실행 후 차트 삽입 | `doc`, `block`, `container` | 없음 | `build_report` → Chart 블록 시 호출<br>내부적으로 `render_chart_to_stream` 사용 |
| 블록 처리 (멀티) | 11 | `multi_table_text` | MultiTable 블록 처리: 소제목 + 요약 텍스트 테이블 | `doc`, `block` | Table 객체 | `build_report` → MultiTable 블록 시 호출 |
| 블록 처리 (멀티) | 12 | `multi_table_chart` | MultiChart 블록 처리: 소제목 + 차트 + 데이터 테이블 | `doc`, `block` | Table 객체 | `build_report` → MultiChart 블록 시 호출<br>내부적으로 `render_chart_to_stream`, `render_table` 호출 |
| 차트 처리 | 13 | `render_chart_to_stream` | 코드(`code_value`) 실행 후 Matplotlib 차트를 PNG 메모리 스트림으로 반환 | `code_value` (str) | `BytesIO` (이미지 스트림) | `add_chart_section`, `multi_table_chart` 내부에서 호출 |
| 보고서 빌드 | 14 | `build_report` | 메인 함수: JSON 데이터 기반으로 블록별 레이아웃 생성 및 삽입 → Word 문서 최종 저장 | `data_dict`, `output_file` | 없음 (docx 저장) | 최상위 함수, 내부에서 `set_default_style` 및 블록별 처리 함수 호출 |

In [2]:
import json
import io
import matplotlib.pyplot as plt
from docx import Document
from docx.oxml import parse_xml
from docx.shared import Pt, Cm, Inches
from docx.oxml.ns import qn
from docx.enum.section import WD_ORIENT
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_ALIGN_VERTICAL
from docx.oxml import OxmlElement

# ============================================================
# 1. JSON 데이터 전처리
# ============================================================
def attach_subtables(data):
    """
    MultiTable / MultiChart 블록에 서브 블록(values) 삽입하고,
    최상위 blocks 리스트에서는 해당 서브 블록 제거
    """
    blocks = data.get("blocks", [])
    block_dict = {blk["id"]: blk for blk in blocks}

    for blk in blocks:
        if blk["type"] in ("MultiTable", "MultiChart"):
            multi_id = blk["id"]
            sub_blocks = [v for k, v in block_dict.items() if k.startswith(multi_id + ".")]
            blk["values"] = sub_blocks

    cleaned_blocks = [
        blk for blk in blocks
        if not any(blk["id"].startswith(multi_id + ".")
                   for multi_id in [b["id"] for b in blocks if b["type"] in ("MultiTable", "MultiChart")])
    ]

    data["blocks"] = cleaned_blocks
    return data


# ============================================================
# 2. 문서 스타일 설정
# ============================================================
def set_default_style(doc):
    """기본 폰트와 페이지 설정"""
    style = doc.styles['Normal']
    font = style.font
    font.name = '맑은 고딕'
    font.size = Pt(10)
    font.element.rPr.rFonts.set(qn('w:eastAsia'), '맑은 고딕')

    pf = style.paragraph_format
    pf.space_before = Pt(0)
    pf.space_after = Pt(0)
    pf.line_spacing = 1

    section = doc.sections[0]
    section.page_width = Cm(21.0)
    section.page_height = Cm(29.7)
    section.orientation = WD_ORIENT.PORTRAIT

    margin = Cm(1.27)
    section.top_margin = margin
    section.bottom_margin = margin
    section.left_margin = margin
    section.right_margin = margin
    section.header_distance = margin
    section.footer_distance = margin


def add_block_heading(target, text, font_name="Malgun Gothic", font_size=14):
    """
    블록 제목 생성
    - target: Document 또는 Cell
    - text: 제목 텍스트
    - font_name: 한글 폰트
    - font_size: pt
    """
    # container가 list/tuple로 넘어오면 첫 셀 사용
    if isinstance(target, (list, tuple)):
        target = target[0]

    para = target.add_paragraph()
    run = para.add_run(text)
    run.bold = True
    run.font.size = Pt(font_size)
    try:
        run.font.name = font_name
        run._element.rPr.rFonts.set(qn('w:eastAsia'), font_name)
    except Exception:
        pass

    # Heading 느낌 여백
    para.paragraph_format.space_before = Pt(6)
    para.paragraph_format.space_after = Pt(4)

    return para

# ============================================================
# 3. 레이아웃 템플릿
# ============================================================
def set_table_autofit(table, autofit=True):
    """
    표를 Word의 '창에 자동 맞춤' 모드처럼 설정하는 함수
    """
    tbl = table._element
    tblPr = tbl.tblPr

    # 기존 tblLayout 제거
    for e in tblPr.findall(qn('w:tblLayout')):
        tblPr.remove(e)

    # <w:tblW> 노드가 없으면 생성
    tblW = tblPr.find(qn('w:tblW'))
    if tblW is None:
        tblW = OxmlElement('w:tblW')
        tblPr.append(tblW)

    if autofit:
        tblW.set(qn('w:type'), 'auto')   # 창에 맞춤
    else:
        tblW.set(qn('w:type'), 'dxa')    # 고정 폭
        tblW.set(qn('w:w'), '9000')      # 적당한 크기 (단위: 1/20 pt)

def create_report_template(doc, layout_type="1col"):
    """
    보고서 레이아웃 템플릿
    layout_type:
        - "1col" : 본문 전체 폭
        - "2col" : 두 개의 컬럼
        - "full_table" : 표/차트 전체 폭
    """
    if layout_type == "1col":
        table = doc.add_table(rows=1, cols=1)
        set_table_autofit(table, autofit=True)
        return table.cell(0, 0)

    elif layout_type == "2col":
        table = doc.add_table(rows=1, cols=2)
        set_table_autofit(table, autofit=True)
        return table.cells  # (왼쪽셀, 오른쪽셀)

    elif layout_type == "full_table":
        table = doc.add_table(rows=1, cols=1)
        set_table_autofit(table, autofit=True)
        return table.cell(0, 0)

    else:
        raise ValueError("지원하지 않는 layout_type 입니다.")

# ============================================================
# 4. 기본 테이블 / 셀 렌더링
# ============================================================
def create_table(doc, n_rows, n_cols, style=None):
    """문서에 n_rows x n_cols 테이블 추가 후 반환"""
    table = doc.add_table(rows=n_rows, cols=n_cols)
    if style:
        table.style = style
    return table

def render_table(values, container, style="Table Grid"):
    """
    리스트[딕셔너리] 기반 표를 Document 또는 Cell에 생성
    - 헤더: 회색 배경 + 굵게 + 가운데 정렬
    - 데이터: 가운데 정렬
    """
    if isinstance(values, str):
        values = json.loads(values)

    if not values:
        return None

    headers = list(values[0].keys())
    if hasattr(container, "add_table"):
        table = container.add_table(rows=1, cols=len(headers))
    else:
        raise ValueError("container must be Document or Cell")
    table.style = style

    # 헤더
    hdr_cells = table.rows[0].cells
    for i, h in enumerate(headers):
        hdr_cells[i].text = str(h)
        hdr_cells[i].paragraphs[0].runs[0].bold = True
        hdr_cells[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
        hdr_cells[i]._tc.get_or_add_tcPr().append(
            parse_xml(
                r'<w:shd xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main" w:fill="D9D9D9"/>'
            )
        )
        hdr_cells[i].vertical_alignment = WD_ALIGN_VERTICAL.CENTER

    # 데이터
    for row in values:
        row_cells = table.add_row().cells
        for i, h in enumerate(headers):
            row_cells[i].text = str(row[h])
            row_cells[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER

    return table


# ============================================================
# 5. 텍스트 / 표 / 차트 블록
# ============================================================
def add_text_section(doc, block, container=None):
    """텍스트 블록 처리 (레이아웃 컨테이너 안에 넣을 수 있음)"""
    target = container if container else doc

    # 제목 처리
    add_block_heading(target, f"{block['id']}. {block['title']}")

    paragraph = target.add_paragraph()
    text = block["value"]
    parts = text.split("**")
    for i, part in enumerate(parts):
        run = paragraph.add_run(part)
        if i % 2 == 1:
            run.bold = True

def add_table_section(doc, block, container=None):
    """Table 블록 처리"""
    target = container if container else doc
 
    # 제목 처리
    add_block_heading(target, f"{block['id']}. {block['title']}")

    render_table(block["values"], target)


def add_chart_section(doc, block, container=None):
    """Chart 블록 처리"""
    target = container if container else doc
 
    # 제목 처리
    add_block_heading(target, f"{block['id']}. {block['title']}")

    img_stream = render_chart_to_stream(block["code_value"])
    paragraph = target.add_paragraph()
    paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    run = paragraph.add_run()
    run.add_picture(img_stream, width=Inches(5.0))


# ============================================================
# 6. 멀티테이블 / 멀티차트 블록
# ============================================================
def multi_table_text(doc, block):
    """MultiTable 블록 처리"""
    # 제목 처리
    add_block_heading(doc, f"{block['id']}. {block['title']}")
    
    child_blocks = block["values"]

    n_cols = len(child_blocks)
    table = create_table(doc, 2, n_cols, style="Table Grid")

    for idx, child in enumerate(child_blocks):
        # 1행: 소제목
        header_cell = table.cell(0, idx)
        header_cell.text = child["title"]
        header_cell._tc.get_or_add_tcPr().append(
            parse_xml(r'<w:shd xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main" w:fill="D9D9D9"/>')
        )
        for p in header_cell.paragraphs:
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER
            for run in p.runs:
                run.font.bold = True

        # 2행: 데이터 요약
        data_cell = table.cell(1, idx)
        values = json.loads(child["values"])
        lines = [f"{k}: {v}" for row in values for k, v in row.items()]
        data_cell.text = "\n".join(lines)

    return table

def multi_table_chart(doc, block):
    """MultiChart 블록 처리"""
    # 제목 처리
    add_block_heading(doc, f"{block['id']}. {block['title']}")
    
    child_blocks = block["values"]
    n_cols = len(child_blocks)
    table = create_table(doc, 3, n_cols)

    # 1행: 소제목
    for j, child in enumerate(child_blocks):
        cell = table.cell(0, j)
        cell.text = child["title"]
        for p in cell.paragraphs:
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER
            if p.runs:
                run = p.runs[0]
                run.font.bold = True
                run.font.size = Pt(9)
        cell._tc.get_or_add_tcPr().append(
            parse_xml(r'<w:shd xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main" w:fill="D9D9D9"/>')
        )

    # 2행: 차트
    for j, child in enumerate(child_blocks):
        cell = table.cell(1, j)
        paragraph = cell.add_paragraph()
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        img_stream = render_chart_to_stream(child["code_value"])
        run = paragraph.add_run()
        run.add_picture(img_stream, width=Inches(2.5))

    # 3행: 데이터 테이블
    for j, child in enumerate(child_blocks):
        cell = table.cell(2, j)
        render_table(child["values"], cell)

    return table


# ============================================================
# 7. 차트 렌더링
# ============================================================
def render_chart_to_stream(code_value):
    """코드 실행 후 차트를 PNG 메모리 스트림으로 반환"""
    exec(code_value, globals())
    fig = plt.gcf()
    img_stream = io.BytesIO()
    fig.savefig(img_stream, format="png", bbox_inches="tight")
    img_stream.seek(0)
    plt.close(fig)
    return img_stream


# ============================================================
# 8. 보고서 빌드
# ============================================================
def build_report(data_dict, output_file="report.docx"):
    """JSON 데이터를 기반으로 보고서 작성"""
    doc = Document()
    set_default_style(doc)

    # 문서 제목
    heading = doc.add_heading(data_dict["doc_title"], level=0)
    heading.paragraph_format.space_before = Pt(0)
    heading.paragraph_format.space_after = Pt(0)

    blocks = data_dict["blocks"]
    for block in blocks:
        btype = block["type"].lower()

        # === 레이아웃 결정 예시 ===
        if btype in ("text", "table"):
            # 본문 전체 폭 레이아웃
            container = create_report_template(doc, "1col")
        elif btype == "chart":
            # 차트는 전체 폭 차지
            container = create_report_template(doc, "full_table")
        else:
            # 멀티형은 독립적으로
            container = None

        # === 블록 삽입 ===
        if btype == "text":
            add_text_section(doc, block, container)
        elif btype == "table":
            add_table_section(doc, block, container)
        elif btype == "chart":
            add_chart_section(doc, block, container)
        elif btype == "multitable":
            multi_table_text(doc, block)
        elif btype == "multichart":
            multi_table_chart(doc, block)

    doc.save(output_file)
    print(f"보고서가 생성되었습니다: {output_file}")


# ============================================================
# 9. 실행
# ============================================================
#if __name__ == "__main__":
file_path = "report_create_data_sample.json"
with open(file_path, "r", encoding="utf-8") as f:
    loaded_data = json.load(f)

converted_data = attach_subtables(loaded_data)
build_report(converted_data, "report_create_result.docx")


보고서가 생성되었습니다: report_create_result.docx
